In [114]:
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report, accuracy_score

from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report
from collections import Counter

In [115]:
df = pd.read_csv('/Users/sararhiger/Desktop/Master thesis /Data/Raw data/Mileparken_measurements.csv', delimiter=";")

df

,recorded,RADON_SHORT_TERM_AVG Bq/m3,PM2_5 μg/m3,CO2 ppm,HUMIDITY %,TEMP °C,VOC ppb,PRESSURE hPa,PM1 μg/m3
0,2025-03-11T00:00:09,NaN,NaN,386.0,31.47,21.34,NaN,NaN,NaN
1,2025-03-11T00:00:11,NaN,3.0,NaN,NaN,NaN,46.0,999.12,3.0
2,2025-03-11T00:05:06,NaN,NaN,405.0,31.47,21.31,NaN,NaN,NaN
3,2025-03-11T00:05:08,NaN,NaN,NaN,NaN,NaN,46.0,999.12,NaN
4,2025-03-11T00:10:10,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
15096,2025-04-03T07:15:25,NaN,NaN,NaN,NaN,NaN,472.0,1028.90,NaN
15097,2025-04-03T07:20:27,NaN,NaN,906.0,26.26,24.38,NaN,NaN,NaN
15098,2025-04-03T07:20:29,NaN,2.0,NaN,NaN,NaN,484.0,1028.94,2.0
15099,2025-04-03T07:25:22,NaN,NaN,901.0,26.56,24.52,NaN,NaN,NaN


### Dataprocessing (Remove NaN and combine rows)

In [116]:
# Remove the Radon column
df.drop(columns=["RADON_SHORT_TERM_AVG Bq/m3"], inplace=True)

# Drop rows where all values (including timestamp) are NaN
df.dropna(how="all", inplace=True)

# Reset index after dropping NaN rows
df.reset_index(drop=True, inplace=True)

# Combine every two consecutive timestamps into one row
combined_rows = []
for i in range(0, len(df) - 1, 2):  # Step by 2 to merge pairs of rows
    combined = df.iloc[i:i+2].ffill().bfill().iloc[-1]  # Fill missing values and take last row
    combined_rows.append(combined)

# Create a new DataFrame with combined timestamps
df_combined = pd.DataFrame(combined_rows)

# Convert the 'recorded' column to datetime format and remove seconds
df_combined["recorded"] = pd.to_datetime(df["recorded"]).dt.strftime("%Y-%m-%d %H:%M")

# Fill NaN values with the value above
df_combined.fillna(method="ffill", inplace=True)

# Change the column names
df_combined.rename(columns={"recorded": "Time"}, inplace=True)

df_combined

,Time,PM2_5 μg/m3,CO2 ppm,HUMIDITY %,TEMP °C,VOC ppb,PRESSURE hPa,PM1 μg/m3
1,2025-03-11 00:00,3.0,386.0,31.47,21.34,46.0,999.12,3.0
3,2025-03-11 00:05,3.0,405.0,31.47,21.31,46.0,999.12,3.0
5,2025-03-11 00:10,3.0,410.0,31.48,21.27,46.0,999.12,3.0
7,2025-03-11 00:15,2.0,400.0,31.49,21.29,46.0,999.20,2.0
9,2025-03-11 00:20,2.0,416.0,31.51,21.28,46.0,999.18,2.0
...,...,...,...,...,...,...,...,...
15091,2025-04-03 07:05,2.0,676.0,25.75,23.90,443.0,1028.92,2.0
15093,2025-04-03 07:10,2.0,791.0,25.96,24.09,443.0,1028.92,2.0
15095,2025-04-03 07:15,3.0,854.0,26.15,24.23,443.0,1028.88,2.0
15097,2025-04-03 07:20,3.0,906.0,26.26,24.38,472.0,1028.90,2.0


### Making hourly values and timestamps 

In [117]:
# Convert timestamps to datetime format
df_combined["Time"] = pd.to_datetime(df_combined["Time"])

# Set the timestamp as index
df_combined.set_index("Time", inplace=True)

# Resample to hourly intervals and take the mean
df_hourly = df_combined.resample("H").mean()

# Reset index to keep timestamp as a column
df_hourly.reset_index(inplace=True)

df_hourly



,Time,PM2_5 μg/m3,CO2 ppm,HUMIDITY %,TEMP °C,VOC ppb,PRESSURE hPa,PM1 μg/m3
0,2025-03-11 00:00:00,2.307692,401.538462,31.523846,21.281538,47.076923,999.221538,2.307692
1,2025-03-11 01:00:00,1.714286,391.928571,31.636429,21.242857,47.642857,999.202857,1.714286
2,2025-03-11 02:00:00,1.307692,412.307692,31.716154,21.182308,46.846154,998.930769,1.307692
3,2025-03-11 03:00:00,2.214286,436.285714,31.790714,21.156429,47.714286,999.007143,2.214286
4,2025-03-11 04:00:00,2.384615,423.615385,31.863846,21.158462,47.461538,999.118462,2.384615
...,...,...,...,...,...,...,...,...
555,2025-04-03 03:00:00,0.769231,488.000000,26.140000,21.586923,355.076923,1028.607692,0.769231
556,2025-04-03 04:00:00,0.928571,521.857143,26.371429,21.496429,360.500000,1028.844286,0.714286
557,2025-04-03 05:00:00,1.307692,499.846154,26.379231,21.770769,359.461538,1028.930769,1.307692
558,2025-04-03 06:00:00,1.071429,561.000000,25.794286,23.004286,411.714286,1028.930000,1.071429


In [118]:
# Load the Excel file
df_GHI = pd.read_excel('/Users/sararhiger/Desktop/Master thesis /Data/Raw data/GHI_Marts.xlsx')

# Convert timestamps to datetime format
df_GHI["Time(utc)"] = pd.to_datetime(df_GHI["Time(utc)"])

# Convert other relevant columns to numeric (if they aren't already)
# Assuming "GHI" is the column with global horizontal irradiance
df_GHI["GHI"] = pd.to_numeric(df_GHI["GHI"], errors='coerce')  # Change "GHI" to your actual column name

# Resample to hourly average
df_GHI = df_GHI.resample("H", on="Time(utc)").mean().reset_index()

df_GHI

,Time(utc),GHI
0,2025-03-11 00:00:00,-715.386333
1,2025-03-11 01:00:00,-888.004500
2,2025-03-11 02:00:00,-1130.582000
3,2025-03-11 03:00:00,-748.845333
4,2025-03-11 04:00:00,-460.084667
...,...,...
499,2025-03-31 19:00:00,66.455833
500,2025-03-31 20:00:00,66.504833
501,2025-03-31 21:00:00,71.803667
502,2025-03-31 22:00:00,65.347000


### Tilføj varmeforbrug, GHI og udetemperatur

In [119]:
df_heat = pd.read_excel('/Users/sararhiger/Desktop/Master thesis /Data/Raw data/mileparken_heat.xlsx')

df_temp = pd.read_excel('/Users/sararhiger/Desktop/Master thesis /Data/Raw data/OutdoorTemp_March2025.xlsx')


df_merged = pd.concat([df_hourly, df_heat["kWh"]], axis=1)

df_merged = pd.concat([df_merged, df_temp["Middel"]], axis=1)

df_merged = pd.concat([df_merged, df_GHI["GHI"]], axis=1)

# Change the column names
df_merged.rename(columns={"Middel": "Outdoor temperature"}, inplace=True)
df_merged.rename(columns={"kWh": "Heating kWh"}, inplace=True)
df_merged.rename(columns={"GHI": "GHI, W/m2"}, inplace=True)

# if Nan, fill with 0
df_merged["GHI, W/m2"] = df_merged["GHI, W/m2"].fillna(0)

df_merged


,Time,PM2_5 μg/m3,CO2 ppm,HUMIDITY %,TEMP °C,VOC ppb,PRESSURE hPa,PM1 μg/m3,Heating kWh,Outdoor temperature,"GHI, W/m2"
0,2025-03-11 00:00:00,2.307692,401.538462,31.523846,21.281538,47.076923,999.221538,2.307692,80.0,3.9,-715.386333
1,2025-03-11 01:00:00,1.714286,391.928571,31.636429,21.242857,47.642857,999.202857,1.714286,60.0,3.9,-888.004500
2,2025-03-11 02:00:00,1.307692,412.307692,31.716154,21.182308,46.846154,998.930769,1.307692,70.0,2.8,-1130.582000
3,2025-03-11 03:00:00,2.214286,436.285714,31.790714,21.156429,47.714286,999.007143,2.214286,70.0,2.6,-748.845333
4,2025-03-11 04:00:00,2.384615,423.615385,31.863846,21.158462,47.461538,999.118462,2.384615,110.0,2.2,-460.084667
...,...,...,...,...,...,...,...,...,...,...,...
555,2025-04-03 03:00:00,0.769231,488.000000,26.140000,21.586923,355.076923,1028.607692,0.769231,50.0,4.9,0.000000
556,2025-04-03 04:00:00,0.928571,521.857143,26.371429,21.496429,360.500000,1028.844286,0.714286,50.0,4.5,0.000000
557,2025-04-03 05:00:00,1.307692,499.846154,26.379231,21.770769,359.461538,1028.930769,1.307692,50.0,3.9,0.000000
558,2025-04-03 06:00:00,1.071429,561.000000,25.794286,23.004286,411.714286,1028.930000,1.071429,80.0,3.5,0.000000


### Add FDD column with implemented faults

In [120]:
# Add a new column with zeros
df_merged["FDD"] = 0

from datetime import datetime

def time_mask(df, start_time: str, end_time: str):
    return (start_time <= df["Time"]) & (df["Time"] <= end_time)

# Tidspunkt 1 - fejl 4 setpunkt 26 grader 
t1, t2 = '2025-03-12 12:00:00', '2025-03-12 15:00:00'
mask = time_mask(df_merged, t1, t2)
df_merged.loc[mask, 'FDD'] = 4

# Tidspunkt 2 - fejl 1 åbent vindue 
t1, t2 = '2025-03-19 08:00:00', '2025-03-19 11:00:00'
mask = time_mask(df_merged, t1, t2)
df_merged.loc[mask, 'FDD'] = 1

# Tidspunkt 3 - fejl 1 åbent vindue 
t1, t2 = '2025-03-20 17:00:00', '2025-03-21 07:00:00'
mask = time_mask(df_merged, t1, t2)
df_merged.loc[mask, 'FDD'] = 1

df_merged

,Time,PM2_5 μg/m3,CO2 ppm,HUMIDITY %,TEMP °C,VOC ppb,PRESSURE hPa,PM1 μg/m3,Heating kWh,Outdoor temperature,"GHI, W/m2",FDD
0,2025-03-11 00:00:00,2.307692,401.538462,31.523846,21.281538,47.076923,999.221538,2.307692,80.0,3.9,-715.386333,0
1,2025-03-11 01:00:00,1.714286,391.928571,31.636429,21.242857,47.642857,999.202857,1.714286,60.0,3.9,-888.004500,0
2,2025-03-11 02:00:00,1.307692,412.307692,31.716154,21.182308,46.846154,998.930769,1.307692,70.0,2.8,-1130.582000,0
3,2025-03-11 03:00:00,2.214286,436.285714,31.790714,21.156429,47.714286,999.007143,2.214286,70.0,2.6,-748.845333,0
4,2025-03-11 04:00:00,2.384615,423.615385,31.863846,21.158462,47.461538,999.118462,2.384615,110.0,2.2,-460.084667,0
...,...,...,...,...,...,...,...,...,...,...,...,...
555,2025-04-03 03:00:00,0.769231,488.000000,26.140000,21.586923,355.076923,1028.607692,0.769231,50.0,4.9,0.000000,0
556,2025-04-03 04:00:00,0.928571,521.857143,26.371429,21.496429,360.500000,1028.844286,0.714286,50.0,4.5,0.000000,0
557,2025-04-03 05:00:00,1.307692,499.846154,26.379231,21.770769,359.461538,1028.930769,1.307692,50.0,3.9,0.000000,0
558,2025-04-03 06:00:00,1.071429,561.000000,25.794286,23.004286,411.714286,1028.930000,1.071429,80.0,3.5,0.000000,0


## Normaliser varmedata

In [121]:
# Find the max value 
max_value = df_merged['Heating kWh'].max()

# Apply division only where the value is not 0
df_merged['Heating kWh'] = np.where(df_merged['Heating kWh'] == 0, 0, df_merged['Heating kWh']/max_value)

df_merged

,Time,PM2_5 μg/m3,CO2 ppm,HUMIDITY %,TEMP °C,VOC ppb,PRESSURE hPa,PM1 μg/m3,Heating kWh,Outdoor temperature,"GHI, W/m2",FDD
0,2025-03-11 00:00:00,2.307692,401.538462,31.523846,21.281538,47.076923,999.221538,2.307692,0.533333,3.9,-715.386333,0
1,2025-03-11 01:00:00,1.714286,391.928571,31.636429,21.242857,47.642857,999.202857,1.714286,0.400000,3.9,-888.004500,0
2,2025-03-11 02:00:00,1.307692,412.307692,31.716154,21.182308,46.846154,998.930769,1.307692,0.466667,2.8,-1130.582000,0
3,2025-03-11 03:00:00,2.214286,436.285714,31.790714,21.156429,47.714286,999.007143,2.214286,0.466667,2.6,-748.845333,0
4,2025-03-11 04:00:00,2.384615,423.615385,31.863846,21.158462,47.461538,999.118462,2.384615,0.733333,2.2,-460.084667,0
...,...,...,...,...,...,...,...,...,...,...,...,...
555,2025-04-03 03:00:00,0.769231,488.000000,26.140000,21.586923,355.076923,1028.607692,0.769231,0.333333,4.9,0.000000,0
556,2025-04-03 04:00:00,0.928571,521.857143,26.371429,21.496429,360.500000,1028.844286,0.714286,0.333333,4.5,0.000000,0
557,2025-04-03 05:00:00,1.307692,499.846154,26.379231,21.770769,359.461538,1028.930769,1.307692,0.333333,3.9,0.000000,0
558,2025-04-03 06:00:00,1.071429,561.000000,25.794286,23.004286,411.714286,1028.930000,1.071429,0.533333,3.5,0.000000,0


## Normaliser temperaturer 

In [122]:
# Find the average temperature 
#mean_value = df_merged['TEMP °C'].mean()

#print(mean_value)

# Substract the mean value from the measured value 

#df_merged['TEMP °C'] = df_merged['TEMP °C'] - mean_value

#df_merged

# # Find the max value 
# max_value = df_merged['TEMP °C'].max()

# # Apply division only where the value is not 0
# df_merged['TEMP °C'] = np.where(df_merged['TEMP °C'] == 0, 0, df_merged['TEMP °C']/max_value)

# Normalize the temperature column from 0 to 1
#df_merged['TEMP °C'] = (df_merged['TEMP °C'] - df_merged['TEMP °C'].min()) / (df_merged['TEMP °C'].max() - df_merged['TEMP °C'].min())

# Normalize the temperature column using z-score normalization
# df_merged['TEMP °C'] = (df_merged['TEMP °C'] - df_merged['TEMP °C'].mean()) / df_merged['TEMP °C'].std()


# df_merged


In [123]:
df_merged.to_csv("/Users/sararhiger/Desktop/Master thesis /Data/Processed data/Mileparken.csv", index=False)


### Plots

In [124]:
fig = go.Figure()

# Plot the temperature curve
fig.add_trace(go.Scatter(
    x=df_merged['Time'], 
    y=df_merged['TEMP °C'], 
    mode='lines', 
    name='Meeting room with implemented faults'
))

# Find timestamps where "FDD" is NOT 0
fault_indices = df_merged['FDD'] != 0
fault_times = df_merged.loc[fault_indices, 'Time']
fault_temps = df_merged.loc[fault_indices, 'TEMP °C']

# Add markers for fault occurrences
fig.add_trace(go.Scatter(
    x=fault_times,
    y=fault_temps,
    mode='markers',
    marker=dict(color='red', size=10, symbol='x'),
    name='Fault Detected (FDD ≠ 0)'
))

# Customize layout
fig.update_layout(
    title="Temperature in the meeting room with faulty conditions",
    xaxis_title="Time",
    yaxis_title="Temperature (°C)",
    legend_title="Legend",
    template="plotly"
)

# Show the plot
fig.show()

In [125]:
fig = go.Figure()

# Plot the temperature curve
fig.add_trace(go.Scatter(
    x=df_merged['Time'], 
    y=df_merged['Heating kWh'], 
    mode='lines', 
    name='Meeting room heating consumption'
))

# Find timestamps where "FDD" is NOT 0
fault_indices = df_merged['FDD'] != 0
fault_times = df_merged.loc[fault_indices, 'Time']
fault_temps = df_merged.loc[fault_indices, 'Heating kWh']

# Add markers for fault occurrences
fig.add_trace(go.Scatter(
    x=fault_times,
    y=fault_temps,
    mode='markers',
    marker=dict(color='red', size=10, symbol='x'),
    name='Fault Detected (FDD ≠ 0)'
))

# Customize layout
fig.update_layout(
    title="Heating consumption for building with faulty conditions",
    xaxis_title="Time",
    yaxis_title="Normalized Heating",
    legend_title="Legend",
    template="plotly"
)

# Show the plot
fig.show()